# LeNet网络
+ 基于mnist数据集
+ 学习量化感知训练 Quantization Aware Trainning

In [1]:
import tensorflow as tf
print(f"tf verion = {tf.__version__}")

import tensorflow_model_optimization as tfmot
from tensorflow.keras.layers import Conv2D,MaxPool2D,Flatten,Dense,Dropout

tf verion = 2.2.0


# 解决GPU内存不足报错，对GPU进行按需分配

In [2]:
from tensorflow.compat.v1 import ConfigProto
from tensorflow.compat.v1 import InteractiveSession

config = ConfigProto()
config.gpu_options.allow_growth = True
session = InteractiveSession(config=config)

## 加载数据集

In [3]:
mnist = tf.keras.datasets.mnist

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

x_train = x_train[..., tf.newaxis]
x_test = x_test[..., tf.newaxis]


## 构建模型

In [4]:
model = tf.keras.models.Sequential([
        Conv2D(filters=6,kernel_size=5,strides=(1,1),padding='same',activation='relu',use_bias=False,input_shape=(28,28,1)),
        MaxPool2D(pool_size=(3,3),strides=2,padding="same"),
        Conv2D(filters=16,kernel_size=5,strides=(1,1),padding='same',activation='relu',use_bias=False),
        MaxPool2D(pool_size=(3,3),strides=2,padding="same"),
        Flatten(input_shape=(7, 7)),
        Dense(120, activation='relu'),
        Dense(84, activation='relu'),
        Dropout(0.2),
        Dense(10, activation='softmax')
    ])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 28, 28, 6)         150       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 14, 14, 6)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 14, 14, 16)        2400      
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 7, 7, 16)          0         
_________________________________________________________________
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
dense (Dense)                (None, 120)               94200     
_________________________________________________________________
dense_1 (Dense)              (None, 84)                1

## 模型训练和评估（普通方式）

In [5]:
def model_run():
    '''
        float32 model
    '''
    print("float32 model:")
    model.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
    print("==> training")
    model.fit(x_train, y_train, epochs=1)
    print("==> evaluate")
    model.evaluate(x_test,  y_test, verbose=2)
    return model



## 模型训练和评估（量化感知训练）

In [6]:
def quant_model_run():
    '''
        quantized model
    '''
    print("quantized model:")
    quant_model = tfmot.quantization.keras.quantize_model(model)

    quant_model.compile(optimizer='adam',
                    loss='sparse_categorical_crossentropy',
                    metrics=['accuracy'])
    print("==> training")
    quant_model.fit(x_train, y_train, epochs=1)
    print("==> evaluate")
    quant_model.evaluate(x_test,  y_test, verbose=2)
    return quant_model

## 运行

In [7]:
if __name__=='__main__':
    normal_model = model_run()
    qat_model = quant_model_run()

float32 model:
==> training
1875/1875 [==============================] - 4s 2ms/step - loss: 0.2013 - accuracy: 0.9381
==> evaluate
313/313 - 2s - loss: 0.0567 - accuracy: 0.9823
quantized model:
==> training
1875/1875 [==============================] - 8s 4ms/step - loss: 0.0690 - accuracy: 0.9796
==> evaluate
313/313 - 1s - loss: 0.0425 - accuracy: 0.9861


## 模型分析

In [8]:
normal_model.save("lenet_normal.hdf5")
normal_model_json = normal_model.to_json()
with open('lenet_normal.json', 'w') as file:
    file.write(normal_model_json)

qat_model.save("lenet_qat.hdf5")
qat_model_json = qat_model.to_json()
with open('lenet_normal.json', 'w') as file:
    file.write(qat_model_json)